**Q1. Summarize for us the goal of this project and how machine learning is useful in trying to accomplish it. As part of your answer, give some background on the dataset and how it can be used to answer the project question. Were there any outliers in the data when you got it, and how did you handle those?  [relevant rubric items: “data exploration”, “outlier investigation”]**

The goal of this project is to identify Enron's POI with real historic dataset, which includes financial data and email data of ~150 Enron employees. Since the dataset is high dimensional and the correlation between features and POI identiy is difficult to reveal with manual exploration, machine learning can be a good tool. This is a typical classification problem with two categories (POI & Non-POI).

There are 146 data points in original dataset (145 people and 1 total). 23 features are included. Besides 'total', no suspicious outliers should be removed, since the extremely high values are likely to be real financial data of Enron's managerial people. 

Missing values are represented by 'NaN', the # of missing values is high for many features, they are going be replaced by zero.

{'to_messages': 59, 'deferral_payments': 107, 'expenses': 51, 'deferred_income': 97, 'email_address': 34, 'from_poi_to_this_person': 59, 'poi_from_rate': 59, 'restricted_stock_deferred': 128, 'shared_receipt_with_poi': 59, 'loan_advances': 142, 'from_messages': 59, 'other': 53, 'director_fees': 129, 'bonus': 64, 'total_stock_value': 20, 'from_this_person_to_poi': 59, 'long_term_incentive': 80, 'restricted_stock': 36, 'salary': 51, 'total_payments': 21, 'poi_to_rate': 59, 'exercised_stock_options': 44}

----

** Q2. What features did you end up using in your POI identifier, and what selection process did you use to pick them? Did you have to do any scaling? Why or why not? As part of the assignment, you should attempt to engineer your own feature that does not come ready-made in the dataset -- explain what feature you tried to make, and the rationale behind it. (You do not necessarily have to use it in the final analysis, only engineer and test it.) In your feature selection step, if you used an algorithm like a decision tree, please also give the feature importances of the features that you use, and if you used an automated feature selection function like SelectKBest, please report the feature scores and reasons for your choice of parameter values.  [relevant rubric items: “create new features”, “properly scale features”, “intelligently select feature”]**

I tried many different combination of features, and finally settled down with 5 key features ('salary', 'expenses', 'exercised_stock_options', 'poi_from_rate', 'poi_to_rate'). The last two ratio features were newly created, since they are better reflection on email behavior than absolute number.

Before conducting formal machine learning, I created some scatterplots to manually observe the correlation among features, and throw promising ones into training. I also use classifiers' attribute (such as DecisionTree's feature_importances and PCA's explained_variance_ratio to judge the importance of features. 

The most important feature in my final algorithm is 'expenses' (0.34964793), and the second one is 'poi_from_rate' (0.28594119).

I created a feature scaling function for repeat use at the beginning. In the final assignment, I didn't adopt it, since it doesn't have an impact on the performance of DecisionTree. I did adopt scaling on other algorithms, such as PCA, because the unit of money and email is different,hence interaction among different features require standardization.

---

** Q3. What algorithm did you end up using? What other one(s) did you try? How did model performance differ between algorithms?  [relevant rubric item: “pick an algorithm”]**

In my final assignment, I used DecisionTreeClassifier. The parameters were tuned with the help of SearchGridCV. 'max_depth', 'min_samples_split' and 'min_samples_leaf' are three important parameters I focused on. Besides, the selection of features also significantly influence the outcome.

I tried many algorithms and test them with K-fold cross validation (adjusted on tester.py). I also used classification_report and confusion_matrix to review outcome.

I explored 6 algorithms in total, their performance and my observations can be found in file 'algorithm_exploration_tyler_jin.ipynb'. A summary here:

- GaussianNB: Not so much parameters to tune for GaussianNB. The choice of features would influence classifier's performance. Feature scaling doesn't make an obvious impact on the performance (which make sense since the algorithm of NB doesn't combine features of different measure unit).
- DecisionTree: The best estimator returned by GridSearchCV varies drastically for different train/test sets and the precision/recall rate is unstable. Too high max_depth usually results in overfitting. With careful tuning, desired performance is achieved (precision >.3, recall>.3).
- RandomForest: Seems random forest doesn't perform better than a single tree. The precision rate is about the same but recall rate significantly drops.
- SVM: Played with different combination of features and parameters, the precision rate is often admirably high while the recall rate is usually painfully low. SVM is conservative and thus fails to identify many positive cases.
- PCA+SVM: The combination seems to be even worse than simply adopting SVM. Probably should not adopt complex algorithm on a relatively small dataset.
- PCA+DecisionTree: The outcome is similiar to PCA+SVM, and the performance didn't exceed adopting DecisionTree only.

---


** Q4. What does it mean to tune the parameters of an algorithm, and what can happen if you don’t do this well?  How did you tune the parameters of your particular algorithm? (Some algorithms do not have parameters that you need to tune -- if this is the case for the one you picked, identify and briefly explain how you would have done it for the model that was not your final choice or a different model that does utilize parameter tuning, e.g. a decision tree classifier).  [relevant rubric item: “tune the algorithm”]**

Machine Learning sometimes can be an art rather than strict science, there is no clear rule about how to set the best parameters for a particular project. The optimal outcome has to be achieved through experiments. In general, tuning is a struggle between bias and variance. If the parameters are not carefully tuned, one of two following consequences are likely to happen: 1. the information in training set is not fully utilized, there is still room to improve performance. 2. the model is overfitted on training set, the performance on test set is ugly. In this project, I mainly rely on SearchGridCV to try different combinations of parameters and find out the best choice. Cross validation is also important to fully utilize the limited data points at hand.

---

**Q5. What is validation, and what’s a classic mistake you can make if you do it wrong? How did you validate your analysis?  [relevant rubric item: “validation strategy”]**

Validation is separating the whole dataset into training and testing sets, train model on one set and test its performance on the other, to get an objective feedback of the real performance of certain model. As mentioned above, if validation is not properly conducted, a seemingly good performance on training set caused by overfitting would appear, and the model will perform badly on other datasets. 

In this project I keep 30% of all data points as test set. I also adopted StratifiedShuffleSplit cross validation with the code from 'tester.py'. I believe this validation method is the best choice for this project for two following reasons:
1. This method creates splits by preserving the same percentage for each target class as in the complete set. In our dataset, # of positive cases is much smaller than negative cases, a random train/test split is likely to result in zero positive case in one of the set. A stratified split method would ensure the existence of positive cases in both sets.
2. The dataset is small, so we must fully utilize every data point in train/test process. StratifiedShuffleSplit method will generate many different sets of train/test split. Every single data point is repeatly used and it can be either training point or test point in each split. Thus our model can be fully trained and tested. 

---

** Q6. Give at least 2 evaluation metrics and your average performance for each of them.  Explain an interpretation of your metrics that says something human-understandable about your algorithm’s performance. [relevant rubric item: “usage of evaluation metrics”]**

The performance of my final submission is tested with tester.py. 

- The accuracy is 0.86886: among all test data points, 86.9% of them were correctly classified)
- The precision is 0.54795: among all people predicted as POI, 54.8% of them were truly POI
- The recall is 0.46850: among all real POI, 46.9% of them were correctly predicted as POI

---